In [ ]:
# Imports
import os
import re
import json
import subprocess
from pathlib import Path
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough # RunnablePassthrough is used when you want to pass the input as it is.
from langchain_core.output_parsers import StrOutputParser # the output from llm has lot of info so to get only the correct content
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

In [ ]:
#setup LLM
llm = ChatOllama(model="mistral-nemo:12b-instruct-2407-q8_0")

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
print(conversational_rag_chain.invoke(
    {"input": "what are the best supported platforms?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"])

In [ ]:
dirname = Path("output")

In [ ]:
dirname.mkdir(parents=True, exist_ok=True)

In [ ]:
# Make Git ignore the global configurtion files
os.environ['GIT_CONFIG_SYSTEM'] = "/dev/null"
os.environ['GIT_CONFIG_GLOBAL'] = "/dev/null"

In [ ]:
subprocess.run(("git", "init", "--initial-branch=main"), cwd=dirname, check=True)

In [ ]:
_ = subprocess.run(("git", "config", "user.name", "TODO"), cwd=dirname, check=True)

In [ ]:
_ = subprocess.run(("git", "config", "user.email", ""), cwd=dirname, check=True)

In [ ]:
# TODO: don't hardcode
#project_name = "findtow" # https://git.sr.ht/~martijnbraam/findtow
#project_name = "gResistor" # https://github.com/stethewwolf/gResistor
project_name = "postmarketos-tweaks" # https://gitlab.postmarketos.org/postmarketOS/postmarketos-tweaks

In [ ]:
_ = subprocess.run(("git", "checkout", "--orphan", project_name), cwd=dirname, check=True)
_ = subprocess.run(("git", "rm", "-rf", "."), cwd=dirname, check=False) # Ignore failures that happen when there are no files

In [ ]:
# Now please run nix-init inside of the output directory
# TODO: pin Nixpkgs revision to the one used by this flake

In [ ]:
nix_file = dirname / "default.nix"
if not os.path.isfile(nix_file):
    if not os.path.isfile("default.nix"):
        raise Exception("default.nix not found, please run nix-init inside of the output directory.")
    else:
        raise Exception("Make sure to run nix-init inside of the output directory, not inside the root of the project!")

In [ ]:
subprocess.run(("git", "add", "--all"), cwd=dirname, check=True)

In [ ]:
subprocess.run(("git", "commit", "-m", "Run nix-init"), cwd=dirname, check=True)

In [ ]:
# TODO: run nix-format after every step

In [ ]:
# TODO: debug Nix why this has no effect when --log-format is internal-json
os.environ['NO_COLOR'] = ""

In [ ]:
build_result = subprocess.run(("nix-build", "--no-out-link", "--log-format", "internal-json", "-E", "(import <nixpkgs> { }).callPackage ./. { }"), capture_output=True, encoding='utf-8', cwd=dirname, check=False)

In [ ]:
# Analyze output

ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

if build_result.returncode == 0:
    print("The nix-build command succeeded 🎉")
else:
    print("The nix-build command failed, let's see what the error was.")
    print(build_result.stdout)
    for line in build_result.stderr.split("\n"):
        if line == "":
            continue
        prefix = "@nix "
        assert line.startswith(prefix), f"Line does not start with @nix : {line}"
        line = line.lstrip(prefix)
        parsed = json.loads(line)
        assert "action" in parsed, f"Nix JSON output does not have an action key: {parsed}"
        if parsed["action"] != "msg":
            continue
        assert "msg" in parsed, f"Nix JSON output does not have a msg key: {parsed}"
        if parsed["msg"].startswith("fetching path input "):
            continue
        #print(parsed)
        if "raw_msg" in parsed:
            error_message = parsed["raw_msg"]
            derivation_path_search = re.search('For full logs, run:\n  nix log (/nix/store/.*.drv)', error_message)
            if derivation_path_search:
                derivation_path = derivation_path_search.group(1)
                # TODO: teach nix log to respect the NO_COLOR environment variable
                error_message = subprocess.run(("nix", "log", derivation_path), capture_output=True, encoding='utf-8', cwd=dirname, check=True).stdout
                # From https://stackoverflow.com/a/14693789
            error_message = ansi_escape.sub('', error_message)
            print(error_message)


# TODO: find instances of the absolute path to the current directory and rewrite them
# TODO: special-case errors like evaluation aborted with the following error message:
# 'lib.customisation.callPackageWith: Function called without required argument "libblkid" at /home/u/Dokumente/repos/RAG-with-Langchain/output/default.nix:8'
# to provide some help to the LLM
# TODO: validate syntax and help the model if it is wrong
# TODO: detect if the model ever outputs the same diff twice and tell the model
# TODO: detect if the error message is the same as before and tell the model that the suggestion did not fix the error
# TODO: take existing derivations from Nixpkgs and modify them, e.g. by deleting lines. Then fine-tune an LLM with the error messages
# TODO: prompt the model to tell if thinks that a dependency is missing and then use something like search.nixos.org but lokal (maybe nix-index) to search for it, then tell the model which dependency to add
# TODO: Run multiple different models at each step and compare
# TODO: detect changes that the LLM should not make, such as changing the hash and revert them
# TODO: if the model does not support reasoning, first ask the model for suggestions on fixing the error, then asking it to implement those suggestions
